In [1]:
import sqlite3
from datetime import datetime, timedelta
from prettytable import PrettyTable

In [1]:
def query_netflows_all_chains():
    # Connect to the database
    conn = sqlite3.connect('chain_data.db')
    cursor = conn.cursor()
    
    # List of chains to query
    chains = ['Ethereum', 'Solana', 'Base', 'Avalanche', 'Optimism']

    # Prepare the table to display the results
    table = PrettyTable(title="Total Inflows/Outflows Across Chains")
    table.field_names = ["Chain", "One Day Net Flow", "Seven Day Net Flow", "Thirty Day Net Flow", "Sixty Day Net Flow"]
    
    for chain in chains:
        # Determine the most recent date for the current chain
        cursor.execute("SELECT MAX(date) FROM net_flows WHERE chain_name = ?", (chain,))
        most_recent_date = cursor.fetchone()[0]
        if not most_recent_date:
            print(f"No data for {chain}.")
            continue

        # Query to calculate the net flows based on the most recent date
        query = f"""
        SELECT
            ROUND(SUM(CASE WHEN date = '{most_recent_date}' THEN net_inflow ELSE 0 END), 2) AS one_day_net_flow,
            ROUND(SUM(CASE WHEN date >= DATE('{most_recent_date}', '-6 day') THEN net_inflow ELSE 0 END), 2) AS seven_day_net_flow,
            ROUND(SUM(CASE WHEN date >= DATE('{most_recent_date}', '-29 day') THEN net_inflow ELSE 0 END), 2) AS thirty_day_net_flow,
            ROUND(SUM(CASE WHEN date >= DATE('{most_recent_date}', '-59 day') THEN net_inflow ELSE 0 END), 2) AS sixty_day_net_flow
        FROM net_flows
        WHERE chain_name = ? AND date <= '{most_recent_date}'
        """
        cursor.execute(query, (chain,))
        row = cursor.fetchone()

        # Format the output row with thousands separators and add to the table
        formatted_row = [chain] + ['{:,}'.format(val) if val else '0' for val in row]
        table.add_row(formatted_row)

    # Close the database connection
    conn.close()
    
    # Print the table
    print(table)

query_netflows_all_chains()



NameError: name 'sqlite3' is not defined

In [22]:
def query_stablecoins_and_changes():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    chains = ['Ethereum', 'Solana', 'Base', 'Avalanche', 'Optimism', 'Sui']
    
    # Tables for displaying results
    counts_table = PrettyTable(["Chain", "Most Recent Count", "7 Day Change", "30 Day Change", "60 Day Change"], title = 'Stablecoin Counts & Changes')
    percent_change_table = PrettyTable(["Chain", "7 Day % Change", "30 Day % Change", "60 Day % Change"], title = 'Stablecoin % Changes')
    
    for chain in chains:
        cursor.execute("SELECT date, total_stables FROM Stablecoins WHERE chain_name = ? ORDER BY date DESC", (chain,))
        rows = cursor.fetchall()
        
        if not rows:
            print(f"No stablecoin data for {chain}.")
            continue
        
        most_recent_count = rows[0][1]
        dates_counts = {row[0]: row[1] for row in rows}
        
        # Calculate changes and percent changes
        changes = {}
        for days in [7, 30, 60]:
            past_date = (datetime.strptime(rows[0][0], '%Y-%m-%d') - timedelta(days=days)).strftime('%Y-%m-%d')
            past_count = dates_counts.get(past_date, most_recent_count)  # Use most recent count if no exact match
            change = most_recent_count - past_count
            percent_change = (change / past_count) * 100 if past_count else 0
            changes[days] = (change, percent_change)
        
        # Add rows to the tables
        counts_table.add_row([chain, f'{most_recent_count:,.2f}', f'{changes[7][0]:,.2f}', f'{changes[30][0]:,.2f}', f'{changes[60][0]:,.2f}'])
        percent_change_table.add_row([chain, f'{changes[7][1]:.2f}%', f'{changes[30][1]:.2f}%', f'{changes[60][1]:.2f}%'])
    
    conn.close()
    
    # Print the tables
    print(counts_table)
    print(percent_change_table)

query_stablecoins_and_changes()


+--------------------------------------------------------------------------------------+
|                             Stablecoin Counts & Changes                              |
+-----------+-------------------+----------------+------------------+------------------+
|   Chain   | Most Recent Count |  7 Day Change  |  30 Day Change   |  60 Day Change   |
+-----------+-------------------+----------------+------------------+------------------+
|  Ethereum | 71,618,068,502.02 | 644,114,102.95 | 1,894,443,655.38 | 4,189,448,967.52 |
|   Solana  |  2,216,709,057.63 | 17,107,131.29  |  286,862,639.12  |  381,239,514.74  |
|    Base   |   303,868,929.41  | -6,491,396.19  |   7,482,268.76   |  -12,118,019.86  |
| Avalanche |  1,261,580,411.44 | 72,803,016.82  |  71,484,209.67   | 1,259,998,878.30 |
|  Optimism |   625,856,234.47  | 16,627,875.62  |  19,771,746.90   |       0.00       |
|    Sui    |   311,037,358.25  | 302,273,380.55 |  311,037,358.25  |  311,037,358.25  |
+-----------+--------

In [23]:
# ETH, SOL, Base, SEI, Sui, INJ, Avalanche, Optimism, 

def query_tvl_and_changes():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    chains = ['Ethereum', 'Solana', 'Base', 'Avalanche', 'Optimism', 'Sui','Sei','Injective']
    
    # Tables for displaying results
    counts_table = PrettyTable(["Chain", "Most Recent Count", "7 Day Change", "30 Day Change", "60 Day Change"], title = 'Total Value Locked & Changes')
    percent_change_table = PrettyTable(["Chain", "7 Day % Change", "30 Day % Change", "60 Day % Change"], title = 'Total Value Locked % Changes')
    
    for chain in chains:
        cursor.execute("SELECT date, TVL FROM TVL WHERE chain_name = ? ORDER BY date DESC", (chain,))
        rows = cursor.fetchall()
        
        if not rows:
            print(f"No TVL data for {chain}.")
            continue
        
        most_recent_count = rows[0][1]
        dates_counts = {row[0]: row[1] for row in rows}
        
        # Calculate changes and percent changes
        changes = {}
        for days in [7, 30, 60]:
            past_date = (datetime.strptime(rows[0][0], '%Y-%m-%d') - timedelta(days=days)).strftime('%Y-%m-%d')
            past_count = dates_counts.get(past_date, most_recent_count)  # Use most recent count if no exact match
            change = most_recent_count - past_count
            percent_change = (change / past_count) * 100 if past_count else 0
            changes[days] = (change, percent_change)
        
        # Add rows to the tables
        counts_table.add_row([chain, f'{most_recent_count:,.2f}', f'{changes[7][0]:,.2f}', f'{changes[30][0]:,.2f}', f'{changes[60][0]:,.2f}'])
        percent_change_table.add_row([chain, f'{changes[7][1]:.2f}%', f'{changes[30][1]:.2f}%', f'{changes[60][1]:.2f}%'])
    
    conn.close()
    
    # Print the tables
    print(counts_table)
    print(percent_change_table)

query_tvl_and_changes()

+------------------------------------------------------------------------------------------+
|                               Total Value Locked & Changes                               |
+-----------+-------------------+------------------+-------------------+-------------------+
|   Chain   | Most Recent Count |   7 Day Change   |   30 Day Change   |   60 Day Change   |
+-----------+-------------------+------------------+-------------------+-------------------+
|  Ethereum | 46,515,049,694.42 | 4,082,492,148.29 | 14,002,018,601.26 | 17,017,253,352.93 |
|   Solana  |  2,056,970,781.86 |  105,331,870.84  |   740,724,935.31  |   736,917,578.83  |
|    Base   |   406,448,270.93  |  10,403,588.73   |   14,381,055.64   |   -41,713,952.82  |
| Avalanche |   921,403,027.28  |  63,725,690.57   |   110,864,644.85  |        0.00       |
|  Optimism |   859,809,350.13  |  71,353,660.35   |   22,987,785.91   |        0.00       |
|    Sui    |   594,603,400.28  |   3,926,541.14   |   266,907,405.09 

In [20]:
def query_dex_volume():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Query the Dex_volume table
    cursor.execute("SELECT * FROM Dex_volume")
    rows = cursor.fetchall()
    
    # Close the connection to the database
    conn.close()
    
    # Create a PrettyTable object with appropriate column names
    dex_volume_table = PrettyTable(["Chain", "24H Volume", "7D Volume", "30D Volume"])
    
    # Iterate over the rows and add them to the table
    for row in rows:
        chain, volume_24h, volume_7d, volume_30d = row
        dex_volume_table.add_row([
            chain, 
            f'{volume_24h:,.2f}',  # Format the number with commas and round to two decimal places
            f'{volume_7d:,.2f}', 
            f'{volume_30d:,.2f}'
        ])
    
    # Print the table
    print(dex_volume_table)

# Call the function to display the table
query_dex_volume()


+-----------+------------------+------------------+-------------------+
|   Chain   |    24H Volume    |    7D Volume     |     30D Volume    |
+-----------+------------------+------------------+-------------------+
|  Ethereum | 1,042,563,983.00 | 9,411,194,654.00 | 35,418,898,041.99 |
|   Solana  |  581,606,226.00  | 5,023,558,299.07 | 21,194,712,583.80 |
|    Base   |  27,501,045.00   |  218,375,790.00  |   852,370,092.43  |
|    Sei    |   1,638,315.00   |  11,491,397.00   |   37,392,536.13   |
|    Sui    |  57,223,518.00   |  481,929,692.00  |  2,437,711,294.94 |
| Injective |  18,064,731.00   |  158,911,276.00  |   594,629,004.53  |
| Avalanche |  78,180,304.00   |  682,268,981.00  |  2,437,630,631.68 |
|  Optimism |  14,750,983.00   |  135,408,352.00  |   495,986,335.94  |
+-----------+------------------+------------------+-------------------+
